<a href="https://colab.research.google.com/github/doodeck/popes-spark/blob/main/Popes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Complete list of Popes Data Analysis

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 126111 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u452-ga~us1-0ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u452-ga~us1-0

In [2]:
!(mkdir data ; cd data ; rm popes.csv ; wget "https://raw.githubusercontent.com/doodeck/popes-spark/main/data/popes.csv" )
!ls -ltraR data/

rm: cannot remove 'popes.csv': No such file or directory
--2025-06-15 05:48:02--  https://raw.githubusercontent.com/doodeck/popes-spark/main/data/popes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38169 (37K) [text/plain]
Saving to: ‘popes.csv’

popes.csv           100%[===================>]  37.27K  --.-KB/s    in 0.02s   

2025-06-15 05:48:02 (2.24 MB/s) - ‘popes.csv’ saved [38169/38169]

data/:
total 48
drwxr-xr-x 1 root root  4096 Jun 15 05:48 ..
drwxr-xr-x 2 root root  4096 Jun 15 05:48 .
-rw-r--r-- 1 root root 38169 Jun 15 05:48 popes.csv


In [3]:
from pyspark.sql import (SparkSession, functions as F)

# Creates a session on a local master
spark = SparkSession.builder.appName("Orders analytics") \
    .master("local[*]").config("spark.ui.port", "4050").getOrCreate()
spark

In [7]:
!curl https://loca.lt/mytunnelpassword

34.106.43.250

In [4]:
# set up a tunnel to spark UI
!node --version
cmd = 'npx localtunnel --port 4050 > tunnel.txt &'
get_ipython().system_raw(cmd)
!sleep 10
!cat tunnel.txt

v20.19.0
your url is: https://orange-kids-taste.loca.lt


In [5]:
from pyspark.sql.types import (StructType, StructField,
                               StringType, IntegerType)

def main(spark):
    path = "./data/"
    filename = "popes.csv"

    # describe schema
    schema = StructType([StructField('pontificate', StringType(), False),
                        StructField('startYear', IntegerType(), False),
                        StructField('endYear', IntegerType(), False),
                        StructField('yearsAndDays', StringType(), False),
                        StructField('days', IntegerType(), False),
                        StructField('engName', StringType(), False),
                        StructField('regName', StringType(), False),
                        StructField('persName', StringType(), False),
                        StructField('placeBirth', StringType(), False),
                        StructField('countryBirth', StringType(), False),
                        StructField('ageStart', IntegerType(), False),
                        StructField('ageEnd', IntegerType(), False),
                        StructField('comments', StringType(), False)])

    # Reads a CSV file with header, called orders.csv,
    # stores it in a dataframe
    df = spark.read.format("csv") \
        .option("header", True) \
        .schema(schema) \
        .load(path + filename)

    # df.cache() # creates an entry in Spark storage
    df.show()
    df.printSchema()

    #purify the column names

    # Analyse with SQL

    df.createOrReplaceTempView("popes")

    sql_df = spark.sql('''
      desc popes
    ''')
    sql_df.show(20)


# setting log level, update this as per your requirement
spark.sparkContext.setLogLevel("warn")

main(spark)


+--------------------+---------+-------+--------------------+-----+--------------------+-----------+--------------------+--------------------+------------+--------+------+--------------------+
|         pontificate|startYear|endYear|        yearsAndDays| days|             engName|    regName|            persName|          placeBirth|countryBirth|ageStart|ageEnd|            comments|
+--------------------+---------+-------+--------------------+-----+--------------------+-----------+--------------------+--------------------+------------+--------+------+--------------------+
|          30 - 64/67|       30|     64|      34 to 37 years|13505|           St. Peter|     PETRUS|Simon Peter, Sime...|  Bethsaida, Galilea|      Israel|    NULL|  NULL|                NULL|
| 64/67(?) - 76/79(?)|       64|     76|            12 years| 4380|           St. Linus|      LINUS|               Linus|Tuscia (Central T...|       Italy|    NULL|  NULL|                NULL|
|       76/79(?) - 88|       76|   

In [6]:
    sql_df = spark.sql("""
      select count(*) as Number_of_records, max(days) from popes
      -- select max(days) from popes
    """)
    sql_df.show(20)

    sql_df = spark.sql("""
      select * from popes where days = (select max(days) from popes)
    """)
    sql_df.show(20)

    sql_df = spark.sql("""
      select * from popes where days = (select min(days) from popes where days > 0)
    """)
    sql_df.show(20)

    sql_df = spark.sql("""
      select * from popes where countryBirth like 'Poland'
    """)
    sql_df.show(20)

    sql_df = spark.sql("""
      select * from popes where ageStart = (select min(ageStart) from popes where ageStart > 0)
    """)
    sql_df.show(20)


+-----------------+---------+
|Number_of_records|max(days)|
+-----------------+---------+
|              272|    13505|
+-----------------+---------+

+-----------+---------+-------+--------------+-----+---------+-------+--------------------+------------------+------------+--------+------+--------+
|pontificate|startYear|endYear|  yearsAndDays| days|  engName|regName|            persName|        placeBirth|countryBirth|ageStart|ageEnd|comments|
+-----------+---------+-------+--------------+-----+---------+-------+--------------------+------------------+------------+--------+------+--------+
| 30 - 64/67|       30|     64|34 to 37 years|13505|St. Peter| PETRUS|Simon Peter, Sime...|Bethsaida, Galilea|      Israel|    NULL|  NULL|    NULL|
+-----------+---------+-------+--------------+-----+---------+-------+--------------------+------------------+------------+--------+------+--------+

+--------------------+---------+-------+------------+----+---------+----------------+------------------